In [2]:
!pip uninstall scipy -y
!pip install scipy==1.10.0
!pip install Gpy GPyOpt

Found existing installation: scipy 1.13.1
Uninstalling scipy-1.13.1:
  Successfully uninstalled scipy-1.13.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 7.2 MB/s eta 0:00:00
  Created wheel for GPyOpt: filename=GPyOpt-1.2.6-py3-none-any.whl size=83602 sha256=d0fae4d27a06754b4ddb98587789c1603e00f6ea4df46ef5884d548b7119244c
  Stored in directory: /root/.cache/pip/wheels/72/17/52/9d818b4c60f733bf49d5cf82bc2758ebbdc57a0471137c37be
Successfully built GPyOpt


In [ ]:
import os
os.kill(os.getpid(), 9)


In [2]:
import pandas as pd
import numpy as np
import GPy
import GPyOpt
from google.colab import drive
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

# Mount Google Drive
drive.mount('/content/drive')

# Load the dataset
file_path = '/content/drive/MyDrive/data.csv'
data = pd.read_csv(file_path)
data = pd.read_csv(file_path)
data=data[['TRANS DATE','MERCHANT NAME', 'CARD NUMBER', 'ORIGINAL GROSS AMT']]
#data= data[data['ORIGINAL GROSS AMT'] < 500]
data= data[data['ORIGINAL GROSS AMT'] > 0]

# Drop rows with any remaining missing values
data['TRANS DATE']=pd.to_datetime(data['TRANS DATE'], errors='coerce')
data_cleaned = data.dropna()

# Convert 'TRANS DATE' to datetime format and handle missing values in 'MERCHANT NAME'
data['TRANS DATE'] = pd.to_datetime(data['TRANS DATE'], errors='coerce')
data= data.loc[data['TRANS DATE']>'01-01-2020']
data['MERCHANT NAME'] = data['MERCHANT NAME'].fillna('Unknown')

Q1 = data['ORIGINAL GROSS AMT'].quantile(0.25)
Q3 = data['ORIGINAL GROSS AMT'].quantile(0.75)

# Calculate the IQR (Interquartile Range)
IQR = Q3 - Q1

# Define the outlier bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out the outliers
data= data[(data['ORIGINAL GROSS AMT'] >= lower_bound) & (data['ORIGINAL GROSS AMT'] <= upper_bound)]


data['P ORIGINAL GROSS AMT'] = data.groupby('CARD NUMBER')['ORIGINAL GROSS AMT'].shift(1)
data['P TRANS DATE'] = data.groupby('CARD NUMBER')['TRANS DATE'].shift(1)
# Drop rows with any remaining missing values
data_cleaned = data.dropna().copy()

# Feature Engineering: Create time-related features
data_cleaned['P_YEAR'] = data_cleaned['P TRANS DATE'].dt.year
data_cleaned['P_MONTH'] = data_cleaned['P TRANS DATE'].dt.month
data_cleaned['P_DAY'] = data_cleaned['P TRANS DATE'].dt.day
data_cleaned['P_DAY_OF_WEEK'] = data_cleaned['P TRANS DATE'].dt.dayofweek
data_cleaned = data_cleaned.dropna()#.copy()

merchant_counts = data['MERCHANT NAME'].value_counts()
rare_merchants = merchant_counts[merchant_counts < 5].index
data['MERCHANT NAME'] = data['MERCHANT NAME'].replace(rare_merchants, 'Other')


# Encode the categorical variable MERCHANT NAME
le_merchant = LabelEncoder()
data_cleaned['MERCHANT NAME'] = le_merchant.fit_transform(data_cleaned['MERCHANT NAME'])
data_cleaned['P MERCHANT NAME'] = data_cleaned.groupby('CARD NUMBER')['MERCHANT NAME'].shift(1)
data_cleaned = data_cleaned.dropna()

# Splitting data into features and target variables
unique_card_numbers = data_cleaned['CARD NUMBER'].unique()
train_list = []
test_list = []

for card_number, group in data_cleaned.groupby('CARD NUMBER'):
    # Determine the number of rows that will go into the test set (last 20%)
    n_test = int(len(group) * 0.2)

    # Split the group into train and test
    train_list.append(group.iloc[:-n_test])  # All except the last 20%
    test_list.append(group.iloc[-n_test:])   # Last 20%

# Concatenate all the train and test groups into two DataFrames
train_df = pd.concat(train_list)
test_df = pd.concat(test_list)

train_list = []
test_list = []

for card_number, group in data_cleaned.groupby('CARD NUMBER'):
    # Determine the number of rows that will go into the test set (last 20%)
    n_test = int(len(group) * 0.2)

    # Ensure at least one sample is included in the test set if there are enough rows
    if n_test == 0 and len(group) > 1:
        n_test = 1

    # Split the group into train and test
    if n_test > 0:
        train_list.append(group.iloc[:-n_test])  # All except the last 20%
        test_list.append(group.iloc[-n_test:])   # Last 20%
    else:
        train_list.append(group)  # If n_test is 0, put all data in the training set

# Concatenate all the train and test groups into two DataFrames
train_df = pd.concat(train_list)
test_df = pd.concat(test_list)

# Reset index if needed
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)
X_train=train_df[['P_YEAR', 'P_MONTH', 'P_DAY', 'P_DAY_OF_WEEK', 'P MERCHANT NAME','P ORIGINAL GROSS AMT', 'CARD NUMBER']]#
X_test=test_df[['P_YEAR', 'P_MONTH', 'P_DAY', 'P_DAY_OF_WEEK', 'P MERCHANT NAME','P ORIGINAL GROSS AMT', 'CARD NUMBER']]
y_test=test_df['ORIGINAL GROSS AMT']
y_train=train_df['ORIGINAL GROSS AMT']
# Prepare feature matrix X and target vector y
#X = data[['P_YEAR', 'P_MONTH', 'P_DAY', 'P_DAY_OF_WEEK', 'P MERCHANT NAME', 'CARD NUMBER']].values
#y = data['ORIGINAL GROSS AMT'].values

# Split data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Mounted at /content/drive


In [ ]:

# Define the function to optimize
def gpr_optimize(params):
    variance = params[0, 0]
    lengthscale = params[0, 1]

    kernel = GPy.kern.RBF(input_dim=X_train.shape[1],
                          variance=variance,
                          lengthscale=lengthscale)
    model = GPy.models.GPRegression(X_train,
                                    np.array(y_train)[:, None],
                                    kernel)

    model.optimize(messages=False)

    # Predictions
    y_pred, _ = model.predict(X_test)

    # Return the negative mean squared error
     #(since Bayesian optimization minimizes the function)
    return mean_squared_error(y_test, y_pred)

# Set the bounds for hyperparameters: variance and lengthscale
bounds = [
    {'name': 'variance', 'type': 'continuous', 'domain': (1e-2, 10)},
    {'name': 'lengthscale', 'type': 'continuous', 'domain': (1e-2, 10)}
]



In [ ]:
# Run Bayesian Optimization
optimizer = GPyOpt.methods.BayesianOptimization(
    f=gpr_optimize,
    domain=bounds,
    acquisition_type='EI',
    exact_feval=True
)

optimizer.run_optimization(max_iter=50)

# Get the best parameters
best_variance = optimizer.X[np.argmin(optimizer.Y)][0]
best_lengthscale = optimizer.X[np.argmin(optimizer.Y)][1]
print(f"Best Variance: {best_variance}")
print(f"Best Lengthscale: {best_lengthscale}")

# Train the final model with optimized parameters
optimized_kernel = GPy.kern.RBF(input_dim=X_train.shape[1],
                                variance=best_variance,
                                lengthscale=best_lengthscale)
final_model = GPy.models.GPRegression(X_train,
                                      np.array(y_train)[:, None],
      # Convert y_train to a NumPy array before adding a new axis
                                      optimized_kernel)

# Optimize the final model
final_model.optimize(messages=True)

# Make predictions
y_pred, y_var = final_model.predict(X_test)

Best Variance: 5.769384317990501
Best Lengthscale: 10.0


Test Errors

In [ ]:
y_pred, y_var = final_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rsme = mean_squared_error(y_test, y_pred, squared=False)
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')
print(f'Root Mean Squared Error: {rsme}')



Mean Squared Error: 1933.5256742336696
Mean Absolute Error: 33.4851459595145
Root Mean Squared Error: 43.9718736720835


Train Errors

In [ ]:
y_pred_train, y_var = final_model.predict(X_train)

mse_train=mean_squared_error(y_train, y_pred_train)
mae_train=mean_absolute_error(y_train, y_pred_train)
rsme_train=mean_squared_error(y_train, y_pred_train, squared=False)
print(f'Mean Squared Error: {mse_train}')
print(f'Mean Absolute Error: {mae_train}')
print(f'Root Mean Squared Error: {rsme_train}')

Mean Squared Error: 1993.893174300814
Mean Absolute Error: 34.12412368304907
Root Mean Squared Error: 44.653030964323285


SAVE MODEL

In [ ]:
final_model.save_model("amt_gpr.zip")
import pickle

# Save the model
with open('/content/drive/My Drive/gpr_model.pkl', 'wb') as file:
    pickle.dump(final_model, file)

In [ ]:
import joblib


# Save the scaler to Google Drive
joblib.dump(scaler, '/content/drive/MyDrive/scaler_amt.pkl')

['/content/drive/MyDrive/scaler_amt.pkl']